# **Sovits (train)**

Author: Francis Hu

E-mail: francishr@whu.edu.cn

QQ: 2235306122

BILIBILI: https://space.bilibili.com/636704927

Repository: https://github.com/Francis-Komizu/Sovits

# Set up

In [ ]:
!git clone https://github.com/Francis-Komizu/Sovits
!git clone https://github.com/bshall/hubert
%cd Sovits
!pip install -r requirements.txt
!sudo apt-get install espeak -y

In [ ]:
%cd /content/Sovits/monotonic_align
!python setup.py build_ext --inplace
%cd ..

# Prepare dataset

Get your own data to the workspace! The dataset should look like:

```
└───wavs
    ├───dev
    │   ├───LJ001-0001.wav
    │   ├───...
    │   └───LJ050-0278.wav
    └───train
        ├───LJ002-0332.wav
        ├───...
        └───LJ047-0007.wav

```



In [ ]:
# just an example
!cp /content/drive/MyDrive/Soft-VC/AccelSpeech-Full.zip /content/Sovits
!unzip AccelSpeech-Full.zip

# Extract speech units

Utilize the content encoder to extract speech units in the audio. 

For more information, refer to [this repo](https://github.com/bshall/acoustic-model).

In [ ]:
%cd ../hubert
!python3 encode.py soft path/to/wavs/directory path/to/soft/directory --extension .wav
%cd ../Sovits

Then you should make filelists for both your training and validation files. The filelists should look like:

```
path/to/wav|path/to/unit
...

```

You may use the following code to generate them!

In [ ]:
import os

# generator filelists

def take_num(path):
  filename = os.path.split(path)[1].replace('.wav', '')
  return filename

def generate_filelist(wav_dir, unit_dir, save_path):
  pairs = []
  wav_files = os.listdir(wav_dir)
  wav_files.sort(key=take_num)
  unit_files = os.listdir(unit_dir)
  unit_files.sort(key=take_num)
  for (wav_file, unit_file) in zip(wav_files, unit_files):
    wav_path = os.path.join(wav_dir, wav_file)
    unit_path = os.path.join(unit_dir, unit_file)
    pair = f'{wav_path}|{unit_path}'
    pairs.append(pair)

  with open(save_path, 'w', encoding='utf-8') as f:
    for pair in pairs:
      f.write(pair + '\n')

# specifiy your own directory path!
train_wav_dir = 'your wavs directory for training'
val_wav_dir = 'your wavs directory for validation'
train_unit_dir = 'your units directory for training'
val_unit_dir = 'your units directory for validation'

# you may change them but you should also update your configuration file
train_filelist_path = 'filelists/train_filelist.txt'
val_filelist_path = 'filelists/val_filelist.txt'
generate_filelist(train_wav_dir, train_unit_dir, train_filelist_path)
generate_filelist(val_wav_dir, val_unit_dir, val_filelist_path)

# Train model

In [ ]:
!mkdir /content/Sovits/logs
!mkdir /content/Sovits/logs/sovtis_base

In [ ]:
# To load the "latest" model to log directory
!cp /content/drive/MyDrive/Soft-VC/G_tmp.pth /content/Sovits/logs/sovtis_base/G_1000.pth
!cp /content/drive/MyDrive/Soft-VC/D_tmp.pth /content/Sovits/logs/sovtis_base/D_1000.pth

In [ ]:
!python train.py -c configs/sovits.json -m sovtis_base

INFO:sovtis_base:Train Epoch: 147 [32%]
INFO:sovtis_base:[2.1933271884918213, 2.3408265113830566, 6.2159423828125, 20.863235473632812, 1.0626306533813477, 0.8173456788063049, 55600, 0.00019633378788836418]
INFO:sovtis_base:Train Epoch: 147 [84%]
INFO:sovtis_base:[2.820115089416504, 2.159792184829712, 5.202322483062744, 18.17592430114746, 1.080228328704834, 0.6188734173774719, 55800, 0.00019633378788836418]
INFO:sovtis_base:====> Epoch: 147
INFO:sovtis_base:Train Epoch: 148 [37%]
INFO:sovtis_base:[1.4652940034866333, 2.870680332183838, 10.414350509643555, 25.629148483276367, 1.092608094215393, 0.5378222465515137, 56000, 0.00019630924616487812]
INFO:sovtis_base:Saving model and optimizer state at iteration 148 to ./logs/sovtis_base/G_sovtis_base.pth
INFO:sovtis_base:Saving model and optimizer state at iteration 148 to ./logs/sovtis_base/D_sovtis_base.pth
INFO:sovtis_base:Train Epoch: 148 [89%]
INFO:sovtis_base:[2.627272605895996, 2.2031548023223877, 2.5312068462371826, 16.331735610961914

# References

https://github.com/bshall/acoustic-model

https://github.com/jaywalnut310/vits